In [78]:
import os 
import random
from queue import Queue
import urllib.request
import bs4 
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.parse import urldefrag
import urllib.robotparser
import re
import numpy as np
from queue import PriorityQueue
from datetime import datetime
from time import time
import time
import threading

In [79]:
# Crawler Parameters
THREADS=3
BACKQUEUES= 9
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
LISTURL = ['https://docs.oracle.com/en/','https://www.oracle.com/corporate/','https://en.wikipedia.org/wiki/Machine_learning','https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html','https://docs.oracle.com/middleware/jet210/jet/index.html','https://en.wikipedia.org/w/api.php','https://en.wikipedia.org/api/','https://en.wikipedia.org/wiki/Weka_(machine_learning)','http://www.nu.edu.pk/', 'https://web.mit.edu/', 'https://www.stanford.edu/','https://www.harvard.edu/', 'https://www.uio.no/english/','https://nlp.stanford.edu/']
# Add any other global parameters here
TIME=0
VISITED = []

FRONTIER

In [80]:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
class frontier: 
  front_queue = [Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0)]
  back_queue = [Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0),Queue(maxsize=0)]
  back_dict = {}
  heap = PriorityQueue()

  p=0
  def __init__(self):
    p=0
    # add seed URLs into frontqueue for initialization
    self.add_URL(LISTURL)
    for i in range(BACKQUEUES):
     self.add_to_backqueue(self.back_queue[i],i)
    # initial time when bqueue was accessed
     current_time = time.time()
     self.heap.put((current_time,i))


  def add_to_backqueue(self,Q,i):
    print('adding to backqueue')
    while (Q.empty()):
      domain = self.get_URLfrontqueue()
      hostname = self.get_hostname(domain) 

      if hostname not in self.back_dict:
        Q.put(domain)
        self.back_dict [hostname] = i
      elif hostname in self.back_dict:
        self.back_queue[self.back_dict[hostname]].put(domain)
    

    
  def get_URLfrontqueue(self):
    p = []
    p = self.probability()
    Choice = [1,2,3,4,5]
    f_num = np.random.choice(Choice,p = p)
    print('Getting from frontqueue')
    while self.front_queue[f_num-1].empty():
      f_num = np.random.choice(Choice,p = p)

    url = self.front_queue[f_num-1].get()
    print('URL from frontqueue: ', url)
    return url

  def get_URL(self):
    curr_time = time.time()

    t , index = self.heap.get()

    URL = self.back_queue[index].get()
    hostname = self.get_hostname(URL)

    if self.back_queue[index].empty():
      del self.back_dict[hostname]
      self.add_to_backqueue(self.back_queue[index],index)

    if (curr_time - t < WAITTIME):
      if ((curr_time - t) > 0):
        time.sleep((curr_time - t))

    self.heap.put((curr_time,index)) 
    print('from backqueue: ', URL)
    return URL

 


  def get_hostname(self,URL):
    URL = urlparse(URL).netloc 
    return URL

  def probability(self):
    n = FRONTQUEUES
    sum = 0
    sum = n*(n+1)/2

    Probability = []
    for i in range(FRONTQUEUES):
      Probability.append((i+1)/sum)

    return Probability

  def add_URL(self,URLs):
    print("Adding urls to frontqueue")
    for i, word in enumerate(URLs):
      p = self.prioritizer(URLs[i],5)
      self.front_queue[p-1].put(URLs[i])
      print('url added to fq',URLs[i])
    
     

  def prioritizer(self,URL,f):
    return random.randint(1,f)
    


FILTER URLS


In [81]:
# fetch URLs and parse URLs and return the absolute links + save content of html page in file
def processURL(URL):

  saved_links = [] 
  req = urllib.request.Request(URL)
  with urllib.request.urlopen(req) as response:
   page = response.read()
 
  # save response as beautfilsoup object
  soup = bs4.BeautifulSoup(page)
  pretty_soup = soup.prettify()
  
  # now store the data into file and parse URLS
  fp=open('data','w+')

  # store all the text in the file as raw text 
  fp.write(soup.get_text() + '\n\n')

  fp.write('\n\n')  
  fp.close()
  
  # store Parsed links in a list (both relative + absolute)
  for link in soup.find_all("a"):
    saved_links.append(link.get("href"))


  home_page = urlparse(URL).netloc 
  home = 'https://' + home_page


  # convert relative to absolute links 
  for i in range(len(saved_links)):
      saved_links[i] = urljoin(home,saved_links[i])
      saved_links[i] = urldefrag(saved_links[i])[0]
  
  return list(set(saved_links)) , home_page



def filterURL(saved_links,home_page):

  filtered_url = []
  j=0

  home_page = 'http://' + home_page
  robot_URL = urljoin(home_page,'/robots.txt')
  print('this is robot URL link dfd')
  print(robot_URL)

 # fetch robots.txt and filter out the not permissable pages
  req = urllib.request.Request(robot_URL)
  with urllib.request.urlopen(req) as response:
   page = response.read()

  soup = bs4.BeautifulSoup(page)
  fp=open('robot.txt','w+')
  rp = urllib.robotparser.RobotFileParser()
  rp.set_url(robot_URL)
  rp.read()
  rp.crawl_delay("*")
  for i in range(len(saved_links)):
   flag=rp.can_fetch("User-agent: *", saved_links[i])
   if flag:
     filtered_url.append(saved_links[i])
  print('returning filtered urls..')
  print(filtered_url)

  return filtered_url





Define Thread and Thread Task

In [82]:
class threadTask (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
     while (True):
       print ("Starting " + self.name)
       URL=f.get_URL()
       print('url from backqueue: ', URL)
       if URL not in VISITED:
         VISITED.append(URL)
         try:
           saved_links, home_page = processURL(URL)
           filtered_URLS=filterURL(saved_links,home_page)
           print('filtered URLS: ', filtered_URLS)
           f.add_URL(filtered_URLS)
           print('number of pages crawled: ', len(VISITED))
         except:
           continue

      
     # define frontier object to be used by threads
f = frontier()

threads = []

thread1 = threadTask(1, "Thread-1", 1)
thread2 = threadTask(2, "Thread-2", 2)
thread3 = threadTask(3, "Thread-3", 3)

# start execution of threads
thread1.start()
thread2.start()
thread3.start()

# append threads to list
threads.append(thread1)
threads.append(thread2)
threads.append(thread3)

# Thread completion wait
for t in threads:
    t.join()
print ("Exiting Main Thread")



Adding urls to frontqueue
url added to fq https://docs.oracle.com/en/
url added to fq https://www.oracle.com/corporate/
url added to fq https://en.wikipedia.org/wiki/Machine_learning
url added to fq https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html
url added to fq https://docs.oracle.com/middleware/jet210/jet/index.html
url added to fq https://en.wikipedia.org/w/api.php
url added to fq https://en.wikipedia.org/api/
url added to fq https://en.wikipedia.org/wiki/Weka_(machine_learning)
url added to fq http://www.nu.edu.pk/
url added to fq https://web.mit.edu/
url added to fq https://www.stanford.edu/
url added to fq https://www.harvard.edu/
url added to fq https://www.uio.no/english/
url added to fq https://nlp.stanford.edu/
adding to backqueue
Getting from frontqueue
URL from frontqueue:  https://en.wikipedia.org/w/api.php
adding to backqueue
Getting from frontqueue
URL from frontqueue:  https://www.oracle.com/corporate/
adding to backqueue
Getting from frontqueue
URL from frontqueue

KeyboardInterrupt: ignored

Initialize everything